In [1]:
import pandas as pd 
import numpy as np
from keras.utils import to_categorical

/home/ashutosh/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = pd.read_csv('train.csv')

In [3]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = data.drop(['label'], axis = 1)
y = data['label']

In [6]:
X = np.array(X).reshape(len(X),28,28,1)
y = np.array(to_categorical(y))

In [8]:
X.shape


(42000, 28, 28, 1)

In [9]:
y.shape


(42000, 10)

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 16)

In [13]:
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D

In [41]:
model = Sequential()
model.add(Convolution2D(32,3,data_format='channels_last',activation='relu',input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(100))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer = 'adadelta', metrics = ['accuracy'])

In [42]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 100)               540900    
_________________________________________________________________
dropout_7 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                1010      
_________________________________________________________________
activation_6 (Activation)    (None, 10)                0         
Total para

In [55]:
model.fit(X_train, y_train)

Epoch 1/1
29400/29400 [==============================] - 74s 3ms/step - loss: 0.3783 - acc: 0.9711


In [43]:
model.fit(X_train, y_train, epochs = 12, batch_size = 32)

Epoch 1/12
29400/29400 [==============================] - 75s 3ms/step - loss: 9.6059 - acc: 0.4000
Epoch 2/12
29400/29400 [==============================] - 76s 3ms/step - loss: 4.9508 - acc: 0.6858
Epoch 3/12
29400/29400 [==============================] - 73s 2ms/step - loss: 1.8558 - acc: 0.8751
Epoch 4/12
29400/29400 [==============================] - 75s 3ms/step - loss: 1.1458 - acc: 0.9207
Epoch 5/12
29400/29400 [==============================] - 74s 3ms/step - loss: 0.9353 - acc: 0.9344
Epoch 6/12
29400/29400 [==============================] - 74s 3ms/step - loss: 0.7908 - acc: 0.9446
Epoch 7/12
29400/29400 [==============================] - 73s 2ms/step - loss: 0.7389 - acc: 0.9471
Epoch 8/12
29400/29400 [==============================] - 76s 3ms/step - loss: 0.6394 - acc: 0.9531
Epoch 9/12
29400/29400 [==============================] - 79s 3ms/step - loss: 0.5915 - acc: 0.9570
Epoch 10/12
29400/29400 [==============================] - 77s 3ms/step - loss: 0.5348 - acc: 0.9611

In [56]:
model.evaluate(X_test, y_test
                       )

12600/12600 [==============================] - 8s 659us/step


[0.32513051092566253, 0.9751587301587301]

In [21]:
unseen = pd.read_csv('test.csv')

In [23]:
unseen = np.array(unseen).reshape(len(unseen),28,28,1)

In [57]:
results = model.predict(unseen)

In [58]:
Label = []
for result in results:
    Label.append(np.argmax(result))


In [59]:
ImageId = np.arange(1,len(Label)+1)

In [60]:
df = pd.DataFrame({'ImageId':ImageId, 'Label':Label})
df.to_csv('submission_cnn.csv', index = False)